In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor


features = pd.read_csv('data_transform/all.csv')

data = features.fillna(100000000)

remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'Date', 'Point', 'CityName'
				 , 'MeanForThroughDay', 'MeanForMonthDayByPoints'
                 , 'MeanForThroughDayByPoints', 'MeanForThroughWeekByPoints'
                 , 'MeanForYearDayByPoints', u'Prev2Day', u'Prev3Day'
                 , u'Next2Day', u'Next3Day', 'NextExisting21Value'
                 , 'PrevExisting21Value', 'NextExisting14Value'
                 , 'PrevExisting14Value', 'NextExisting3Value'
                 , 'PrevExisting3Value'
                 , 'BranchNumber', 'MeanForMonth', 'Next1Day', 'Prev7Day'
                 ]


dataTrain = data[data.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])]
y_train = dataTrain[['Count']]

dataValidation =  data[data.IsValidation == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])]
y_test = dataValidation[['Count']]

dataResult =  data[data.IsTest == 1]
dataResult = dataResult[dataResult.columns.difference(remove_column)]

X_res = dataResult[dataResult.columns.difference(['Count'])]

'''
dtrain = xgb.DMatrix( X_train, label=y_train)
dtest = xgb.DMatrix( X_test, label=y_test)
dres = xgb.DMatrix( X_res )
evallist  = [(dtest,'eval'), (dtrain,'train')]

param = {'max_depth': 6,
 		'silent': 1,
 		'eta': 0.01,
 		'eval_metric': 'mae'
 }

num_round = 100

bst = xgb.train( param, dtrain, num_round, evallist, verbose_eval=10 )


result = bst.predict(dres)
'''

"\ndtrain = xgb.DMatrix( X_train, label=y_train)\ndtest = xgb.DMatrix( X_test, label=y_test)\ndres = xgb.DMatrix( X_res )\nevallist  = [(dtest,'eval'), (dtrain,'train')]\n\nparam = {'max_depth': 6,\n \t\t'silent': 1,\n \t\t'eta': 0.01,\n \t\t'eval_metric': 'mae'\n }\n\nnum_round = 100\n\nbst = xgb.train( param, dtrain, num_round, evallist, verbose_eval=10 )\n\n\nresult = bst.predict(dres)\n"

In [20]:
X_train.columns

Index(['CityID', 'MeanForMonthByPoints', 'MeanForMonthDay',
       'MeanForThroughMonth', 'MeanForThroughMonthByPoints',
       'MeanForThroughWeek', 'MeanForWeekDay', 'MeanForWeekDayByPoints',
       'MeanForYear', 'MeanForYearByPoints', 'MeanForYearDay', 'Month',
       'MonthDay', 'ThroughDay', 'ThroughMonth', 'ThroughWeek', 'WeekDay',
       'Year', 'YearDay'],
      dtype='object')

In [21]:
gbr = GradientBoostingRegressor(n_estimators=80, loss="lad", verbose=1, max_depth=7, random_state=10)

In [22]:
gbr.fit(X_train, y_train)

/opt/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1          48.1586           15.54s
         2          44.2569           14.60s
         3          40.7485           14.27s
         4          37.6195           13.92s
         5          34.9142           13.60s
         6          32.4959           13.39s
         7          30.3464           13.25s
         8          28.4594           13.06s
         9          26.7705           12.82s
        10          25.2926           12.66s
        20          17.6518           10.75s
        30          15.5902            8.73s
        40          14.9663            6.56s
        50          14.6562            4.63s
        60          14.3835            2.94s
        70          14.1581            1.41s
        80          13.9683            0.00s


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='lad',
             max_depth=7, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=10, subsample=1.0, verbose=1, warm_start=False)

In [23]:
result = gbr.predict(X_test)
mae(result, y_test)

15.584894753194439

In [24]:
result = gbr.predict(X_res)

In [25]:
dataResult.shape

(7337, 20)

In [26]:
1 == True

True

In [27]:
resDataFrame = pd.DataFrame()
resDataFrame["Date"] = data.Date[data.IsTest == True]
resDataFrame.Date = resDataFrame.Date.apply(lambda x: pd.to_datetime(x))
resDataFrame["Point"] = data.Point[data.IsTest == True]
resDataFrame.index = range(resDataFrame.shape[0])
resDataFrame["Count"] = pd.Series(result)

In [28]:
resDataFrame.head()

,Date,Point,Count
0,2011-12-14,Сыктывкар-1,68.582369
1,2011-12-19,Сыктывкар-1,73.229086
2,2011-12-26,Сыктывкар-1,106.566636
3,2011-12-27,Сыктывкар-1,110.311289
4,2012-01-12,Сыктывкар-1,117.976172


In [29]:
resDataFrame.tail()

,Date,Point,Count
7332,2016-09-27,Новоалтайск-1,19.295813
7333,2016-09-27,Энгельс-1,54.387129
7334,2016-09-27,Зеленоград-1,133.524927
7335,2016-09-27,Ленинск-Кузнецкий-1,53.359373
7336,2016-09-27,Самара-3,328.769516


In [30]:
originTest = pd.read_csv("data_original/test.csv")

In [31]:
originTest.Date = originTest.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))

In [32]:
originTest.head()

,Date,Point,Count
0,2016-07-08,Ульяновск-2,NaN
1,2016-07-25,Балашиха-1,NaN
2,2014-11-09,Ухта-1,NaN
3,2015-11-12,Альметьевск-1,NaN
4,2014-07-17,Ухта-1,NaN


In [33]:
resDataFrame[(resDataFrame.Date == originTest.Date[0]) & (resDataFrame.Point == originTest.Point[0]) ]

,Date,Point,Count
6495,2016-07-08,Ульяновск-2,220.701822


In [34]:
resNP = np.array([np.nan] * originTest.shape[0])

for i in originTest.index:
    resNP[i] = resDataFrame.Count[(resDataFrame.Date == originTest.Date[i]) &
                                             (resDataFrame.Point == originTest.Point[i])]

In [35]:
resNP.shape

(7337,)

In [36]:
originTest.Count = pd.Series(resNP)

In [37]:
originTest.Count = originTest.Count.apply(lambda x: 1 if x < 1 else x)

In [38]:
originTest.Count = originTest.Count.apply(lambda x: round(x) if abs(x - round(x)) < 0.15 else x)

In [39]:
originTest['Year'] = originTest.Date.apply(lambda x: x.year)
originTest['Month'] = originTest.Date.apply(lambda x: x.month)
originTest['MonthDay'] = originTest.Date.apply(lambda x: x.day)

In [40]:
OriginalDate = []
k = 9
for i in range(originTest.shape[0]):
    daystr = str(originTest.MonthDay[i]) if originTest.MonthDay[i] >= 10 else  "0" + str(originTest.MonthDay[i])
    monthstr = str(originTest.Month[i]) if originTest.Month[i] >= 10 else  "0" + str(originTest.Month[i])
    OriginalDate.append(daystr + "." + monthstr + "." +  str(originTest.Year[i]))

In [41]:
originTest["OriginalDate"] = pd.Series(OriginalDate)

In [42]:
originTest.tail()

,Date,Point,Count,Year,Month,MonthDay,OriginalDate
7332,2015-05-27,Белгород-1,69.364244,2015,5,27,27.05.2015
7333,2015-01-31,Самара-3,161.732720,2015,1,31,31.01.2015
7334,2015-11-12,Вологда-1,33.829153,2015,11,12,12.11.2015
7335,2016-05-07,Домодедово-1,98.234087,2016,5,7,07.05.2016
7336,2015-04-18,Махачкала-1,97.000000,2015,4,18,18.04.2015


In [43]:
originTest = originTest[["OriginalDate", "Point", "Count"]]

In [44]:
originTest.head()

,OriginalDate,Point,Count
0,08.07.2016,Ульяновск-2,220.701822
1,25.07.2016,Балашиха-1,118.323384
2,09.11.2014,Ухта-1,173.000000
3,12.11.2015,Альметьевск-1,60.378913
4,17.07.2014,Ухта-1,154.427647


In [45]:
originTest.tail()

,OriginalDate,Point,Count
7332,27.05.2015,Белгород-1,69.364244
7333,31.01.2015,Самара-3,161.732720
7334,12.11.2015,Вологда-1,33.829153
7335,07.05.2016,Домодедово-1,98.234087
7336,18.04.2015,Махачкала-1,97.000000


In [46]:
originTest.to_csv('submission.csv', index=None, header=None)

In [47]:
pd.isnull(originTest).sum() > 0

OriginalDate    False
Point           False
Count           False
dtype: bool

In [48]:
originTest

,OriginalDate,Point,Count
0,08.07.2016,Ульяновск-2,220.701822
1,25.07.2016,Балашиха-1,118.323384
2,09.11.2014,Ухта-1,173.000000
3,12.11.2015,Альметьевск-1,60.378913
4,17.07.2014,Ухта-1,154.427647
5,18.04.2016,Сургут-1,191.000000
6,26.08.2015,Волжский-1,82.183014
7,05.02.2015,Майкоп-1,62.413892
8,28.09.2015,Сарапул-1,52.681614
9,23.04.2016,Москва 1-1,118.000000
